In [123]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
from tensorflow import keras
from keras.models import Model
from tensorflow.python.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.callbacks import EarlyStopping

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
symptom_severity = pd.read_csv('/content/drive/MyDrive/CapstoneProjectBangkitMachineLearning/Data/Symptom-severity.csv')
symptom_severity.head()

,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5


In [5]:
dataset = pd.read_csv('/content/drive/MyDrive/CapstoneProjectBangkitMachineLearning/Data/dataset.csv')
dataset.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
symptom_description = pd.read_csv('/content/drive/MyDrive/CapstoneProjectBangkitMachineLearning/Data/symptom_Description.csv')
symptom_description.head()

,Disease,Description
0,Drug Reaction,An adverse drug reaction (ADR) is an injury ca...
1,Malaria,An infectious disease caused by protozoan para...
2,Allergy,An allergy is an immune system response to a f...
3,Hypothyroidism,"Hypothyroidism, also called underactive thyroi..."
4,Psoriasis,Psoriasis is a common skin disorder that forms...


In [7]:
symptom_precaution = pd.read_csv('/content/drive/MyDrive/CapstoneProjectBangkitMachineLearning/Data/symptom_precaution.csv')
symptom_precaution.head()

,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
2,Allergy,apply calamine,cover area with bandage,NaN,use ice to compress itching
3,Hypothyroidism,reduce stress,exercise,eat healthy,get proper sleep
4,Psoriasis,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths


### Urutkan dataframe berdasarkan kolom 'Diseases' secara menurun


In [8]:
dataset = dataset.sort_values(by='Disease', ascending=True)
print(dataset)

                                      Disease    Symptom_1  Symptom_2  \
1186  (vertigo) Paroymsal  Positional Vertigo     vomiting   headache   
2865  (vertigo) Paroymsal  Positional Vertigo     vomiting   headache   
1187  (vertigo) Paroymsal  Positional Vertigo     vomiting   headache   
4136  (vertigo) Paroymsal  Positional Vertigo     vomiting   headache   
1188  (vertigo) Paroymsal  Positional Vertigo     vomiting   headache   
...                                       ...          ...        ...   
2520                              hepatitis A   joint_pain   vomiting   
3914                              hepatitis A   joint_pain   vomiting   
4037                              hepatitis A   joint_pain   vomiting   
3504                              hepatitis A   joint_pain   vomiting   
1016                              hepatitis A   joint_pain   vomiting   

            Symptom_3            Symptom_4          Symptom_5  \
1186           nausea   spinning_movements       unsteadin

### Remove Duplicated Data

In [9]:
dataset = dataset.drop_duplicates(subset='Disease', keep='first')
print(dataset)

                                      Disease             Symptom_1  \
1186  (vertigo) Paroymsal  Positional Vertigo              vomiting   
1295                                     AIDS        muscle_wasting   
2012                                     Acne    pus_filled_pimples   
2976                      Alcoholic hepatitis              vomiting   
2625                                  Allergy   continuous_sneezing   
1991                                Arthritis       muscle_weakness   
3904                         Bronchial Asthma               fatigue   
3292                     Cervical spondylosis             back_pain   
4690                              Chicken pox               itching   
2791                      Chronic cholestasis               itching   
3224                              Common Cold   continuous_sneezing   
179                                    Dengue             skin_rash   
3533                                Diabetes                fatigue   
2339  

In [10]:
df = symptom_description.merge(symptom_precaution.merge(dataset, how='inner', on = 'Disease'), how='inner', on='Disease')
df.head()

,Disease,Description,Precaution_1,Precaution_2,Precaution_3,Precaution_4,Symptom_1,Symptom_2,Symptom_3,Symptom_4,...,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Drug Reaction,An adverse drug reaction (ADR) is an injury ca...,stop irritation,consult nearest hospital,stop taking drug,follow up,itching,skin_rash,stomach_pain,burning_micturition,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Malaria,An infectious disease caused by protozoan para...,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out,chills,vomiting,high_fever,sweating,...,muscle_pain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Allergy,An allergy is an immune system response to a f...,apply calamine,cover area with bandage,NaN,use ice to compress itching,continuous_sneezing,shivering,chills,watering_from_eyes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hypothyroidism,"Hypothyroidism, also called underactive thyroi...",reduce stress,exercise,eat healthy,get proper sleep,fatigue,weight_gain,cold_hands_and_feets,mood_swings,...,enlarged_thyroid,brittle_nails,swollen_extremeties,depression,irritability,abnormal_menstruation,NaN,NaN,NaN,NaN
4,Psoriasis,Psoriasis is a common skin disorder that forms...,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths,skin_rash,joint_pain,skin_peeling,small_dents_in_nails,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Pre-process Data

In [11]:
# Mengganti spaces dengan "_" dan menghapus spaces di bagian kiri kanan string
def substitute_spaces(string):
  if isinstance(string, str):
    string = string.strip()
    #string = re.sub(" ","_",string)
  return string


In [12]:
for cols in dataset.columns:
 dataset[cols] = dataset[cols].apply(substitute_spaces)
for cols in symptom_severity.columns:
  symptom_severity[cols] = symptom_severity[cols].apply(substitute_spaces)

In [13]:
dataset.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
1186,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements,unsteadiness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1295,AIDS,muscle_wasting,patches_in_throat,high_fever,extra_marital_contacts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012,Acne,pus_filled_pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2976,Alcoholic hepatitis,vomiting,yellowish_skin,abdominal_pain,swelling_of_stomach,distention_of_abdomen,history_of_alcohol_consumption,fluid_overload,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2625,Allergy,continuous_sneezing,shivering,chills,watering_from_eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
symptom_severity.head()

,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5


In [15]:
# Mencocokkan dengan data pada symptom_severity
def matched_data(values):
  values = symptom_severity.loc[symptom_severity['Symptom'] == values]['weight']
  if len(values) > 0:
    return values.iloc[0]  # Return the first matching value
  else:
    return 0

In [16]:
cols = ['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4', 'Symptom_5', 
        'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9', 'Symptom_10', 
        'Symptom_11', 'Symptom_12', 'Symptom_13', 'Symptom_14', 'Symptom_15', 
        'Symptom_16', 'Symptom_17']

for i in cols:
  dataset[i] = dataset[i].apply(matched_data)

In [17]:
dataset

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
1186,(vertigo) Paroymsal Positional Vertigo,5,3,5,6,4,0,0,0,0,0,0,0,0,0,0,0,0
1295,AIDS,3,6,7,5,0,0,0,0,0,0,0,0,0,0,0,0,0
2012,Acne,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2976,Alcoholic hepatitis,5,3,4,7,4,5,6,0,0,0,0,0,0,0,0,0,0
2625,Allergy,4,5,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0
1991,Arthritis,2,4,5,5,2,0,0,0,0,0,0,0,0,0,0,0,0
3904,Bronchial Asthma,4,4,7,4,5,4,0,0,0,0,0,0,0,0,0,0,0
3292,Cervical spondylosis,3,7,5,4,4,0,0,0,0,0,0,0,0,0,0,0,0
4690,Chicken pox,1,3,4,2,7,3,4,5,6,6,3,0,0,0,0,0,0
2791,Chronic cholestasis,1,5,3,5,4,4,4,0,0,0,0,0,0,0,0,0,0


## Modeling

In [19]:
data = dataset.copy()

In [20]:
# Bagi data train dan validation

X = data.drop(['Disease'], axis = 1)
y = dataset['Disease']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123)

In [27]:
#jumlah penyakit
jmlh = len(y)
print(jmlh)

41


In [28]:
X_train.shape

(32, 17)

In [75]:
np.array(y_train)

array(['Hypothyroidism', 'Urinary tract infection', 'Hepatitis C',
       'Hypertension', 'Dimorphic hemmorhoids(piles)', 'Chicken pox',
       'Hepatitis D', 'Heart attack', 'Cervical spondylosis',
       'hepatitis A', 'Alcoholic hepatitis', 'Typhoid', 'Hyperthyroidism',
       'GERD', 'Malaria', 'Allergy', 'Migraine', 'Tuberculosis',
       'Hypoglycemia', 'Fungal infection', 'Osteoarthristis',
       'Drug Reaction', '(vertigo) Paroymsal  Positional Vertigo',
       'Chronic cholestasis', 'Paralysis (brain hemorrhage)',
       'Peptic ulcer diseae', 'Hepatitis E', 'Hepatitis B',
       'Gastroenteritis', 'Pneumonia', 'Jaundice', 'Acne'], dtype=object)

In [72]:
X_train

,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
3270,4,3,5,3,2,4,5,6,5,5,3,2,6,0,0,0,0
1617,6,4,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3588,4,3,5,4,4,5,0,0,0,0,0,0,0,0,0,0,0
103,3,7,4,3,0,0,0,0,0,0,0,0,0,0,0,0,0
2339,4,5,6,5,6,0,0,0,0,0,0,0,0,0,0,0,0
4690,1,3,4,2,7,3,4,5,6,6,3,0,0,0,0,0,0
4737,3,5,4,3,4,5,4,4,4,0,0,0,0,0,0,0,0
1114,5,4,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3292,3,7,5,4,4,0,0,0,0,0,0,0,0,0,0,0,0
1420,3,5,3,4,5,4,4,6,5,4,2,0,0,0,0,0,0


In [76]:
y_val

3320            Impetigo
3533            Diabetes
179               Dengue
805            Psoriasis
3224         Common Cold
1991           Arthritis
1295                AIDS
3904    Bronchial Asthma
1946      Varicose veins
Name: Disease, dtype: object

In [77]:
#ubah labeh jadi numerik (int)
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.fit_transform(y_val)

In [138]:
from tensorflow.python.keras.layers import Dense, Dropout

In [153]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape= (X_train.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(41, activation='softmax')
  ])
model.compile(optimizer= 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

#callback
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

In [154]:
train = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data= (X_val, y_val))

Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 4.0177 - accuracy: 0.0625 - val_loss: 3.9116 - val_accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 40ms/step - loss: 3.7670 - accuracy: 0.0938 - val_loss: 3.8856 - val_accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 59ms/step - loss: 3.5493 - accuracy: 0.1250 - val_loss: 3.8773 - val_accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 56ms/step - loss: 3.3612 - accuracy: 0.1875 - val_loss: 3.8802 - val_accuracy: 0.0000e+00
Epoch 5/50
1/1 [==============================] - 0s 40ms/step - loss: 3.1926 - accuracy: 0.2188 - val_loss: 3.8910 - val_accuracy: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 42ms/step - loss: 3.0368 - accuracy: 0.2812 - val_loss: 3.9066 - val_accuracy: 0.0000e+00
Epoch 7/50
1/1 [==============================] - 0s 57ms/step - loss: 2.8922 - accuracy: 0.3438 - val_loss: 3.9274 - val_accuracy: 0.0000e+00
E